In [1]:
# 必要なSparkモジュールの読み込み
from pyspark.sql import SparkSession

# SparkSessionの作成
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
# RDDの生成
# 配列から生成する方法
source = [
    ('Amber', 22),
    ('Alfred', 23),
    ('Skye', 4),
    ('Albert', 12),
    ('Amber', 9),
]
data = sc.parallelize(source)
# ファイルからの読み込み(ファイルデータ：http://tomdrabas.com/data/VS14MORT.txt.gz )
# SparkではGzipなどの圧縮ファイルもそのまま扱える
data_from_file = sc.textFile("/home/jovyan/work/01_入門PySpark/VS14MORT.txt.gz", 4)

In [3]:
print("----------  Source  ----------")
print(data.collect())
print("----------   File   ----------")
print(data_from_file.take(1))
file_row_count = data_from_file.count()
print(file_row_count)

----------  Source  ----------
[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]
----------   File   ----------
['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']
2631171


In [4]:
# RDDにはスキーマが存在しないので、様々なデータをソースにできる
data_heterogenous = sc.parallelize([
    ('Ferrari', 'fast'), # タプル(イミュータブルなリスト型)
    {'Porsche': 100000 }, # ハッシュ
    ('Spaini', 'visited', 4504), # リスト
]).collect()
data_heterogenous[1]['Porsche']

100000

In [5]:
# ファイルから読み込んだデータがわかりずらいのでパースする関数の定義
# ソース: https://github.com/drabastomek/learningPySpark/blob/master/Chapter02/LearningPySpark_Chapter02.ipynb
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]
    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs

# パースを実行する
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [6]:
# 様々な変換処理を実施してみる
data_2014 = data_from_file_conv.map(lambda row: row[16])
print("----- map変換 -----")
print(data_2014.take(10))

data_filtered = data_from_file_conv.filter(lambda row: row[16] == '2014' and row[21] == '0')
print("----- filter変換 -----")
print(str(file_row_count) + " -> " +  str(data_filtered.count())) # こっちも処理に時間がかかる

data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) * 2) )
print("----- flatMap変換 -----")
print(data_2014_flat.take(10))

# カラムを絞り込んでからdistinctをかける
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct() # 処理に時間がかかる -> countと同じ理由かな
print("----- distinct変換 -----")
print(distinct_gender.collect())

# 抽出割合
fraction = 0.1
# 第一引数：サンプリングで置き換えを許容するかのBoolean, 第二引数：抽出比率, 第三引数：乱数シード
data_sample = data_from_file_conv.sample(False, fraction, 666)
print("----- sample変換 -----")
print(str(file_row_count) + " -> " +  str(data_sample.count())) # 処理に時間がかかる

rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c', 10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])
rdd3 = rdd1.leftOuterJoin(rdd2)
print("----- leftOuterJoin変換 -----")
print(rdd3.collect()) # rdd1に存在するキー(第一要素)に対して、各キーに対してvalueのタプルを作成する。'a'が2つ出現しているので、多対多になる？
rdd3 = rdd1.join(rdd2)
print("----- join変換 -----")
print(rdd3.collect()) # 両方に存在するキー(第一要素)に対して、各キーに対してvalueのタプルを作成する。'a'が2つ出現しているので、多対多になる？
# 検証
rdd1 = sc.parallelize([('a', 1), ('a', 1)])
rdd2 = sc.parallelize([('a', 4), ('a', 1)])
rdd3 = rdd1.leftOuterJoin(rdd2)
print("----- leftOuterJoin検証 -----")
print(rdd3.collect()) # 両方存在するキー(第一要素)に対して、各キーに対してvalueのタプルを作成する。'a'が2つ出現しているので、多対多になる？
# 2 * 2 = 4要素になっている(予想通り)

----- map変換 -----
['2014', '2014', '2014', '2014', '2014', '2014', '2014', '2014', '2014', '-99']
----- filter変換 -----
2631171 -> 22
----- flatMap変換 -----
['2014', 4028, '2014', 4028, '2014', 4028, '2014', 4028, '2014', 4028]
----- distinct変換 -----
['M', 'F', '-99']
----- sample変換 -----
2631171 -> 263247
----- leftOuterJoin変換 -----
[('b', (4, '6')), ('a', (1, 4)), ('a', (1, 1)), ('c', (10, None))]
----- join変換 -----
[('b', (4, '6')), ('a', (1, 4)), ('a', (1, 1))]
----- leftOuterJoin検証 -----
[('a', (1, 4)), ('a', (1, 1)), ('a', (1, 4)), ('a', (1, 1))]


In [7]:
# 様々なアクションを試す(すでに実施済みのものもあるが一応)
# データ量が多いと処理に時間がかかるので、dataに対して実施する
print("----- take アクション -----")
print(data.take(2))
print("----- collect アクション -----")
print(data.collect())
print("----- reduce アクション -----")
print(data.map(lambda row: row[1]).reduce(lambda x, y: x + y))
print("----- count アクション -----")
print(data.count())
print("----- saveAsTextFile アクション : before -----")
%ls sample
data.saveAsTextFile("sample")
print("----- saveAsTextFile アクション : after -----")
%ls sample
%rm -r sample
print("----- foreach アクション -----")
data.foreach(lambda x: print(x)) # notebookに出力するわけではないので出力はない…らしい。

----- take アクション -----
[('Amber', 22), ('Alfred', 23)]
----- collect アクション -----
[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]
----- reduce アクション -----
70
----- count アクション -----
5
----- saveAsTextFile アクション : before -----
ls: cannot access 'sample': No such file or directory
----- saveAsTextFile アクション : after -----
part-00000  part-00003  part-00006  part-00009  _SUCCESS
part-00001  part-00004  part-00007  part-00010
part-00002  part-00005  part-00008  part-00011
----- foreach アクション -----


CLIで実行した結果。実行タイミングによって処理順が異なることがわかる。
```bash
>>> data.foreach(lambda x: print(x))
('Amber', 9)                                                      (0 + 12) / 12]
('Albert', 12)
('Skye', 4)
('Amber', 22)
('Alfred', 23)
>>> data.foreach(lambda x: print(x))
('Alfred', 23)
('Amber', 9)
('Albert', 12)
('Amber', 22)
('Skye', 4)
```